***
# TP4 : Filtrage (continuation)
***

**Plan :**

# I. Filtrage en fréquence

Le traitement local d’une image  utilise la notion de filtre. Ces filtres peuvent s’écrire sous la forme d’un produit de convolution entre une matrice (ou noyau de convolution) et d’une image. Malheureusement,  en  appliquant  de  gros  masques  de  convolution,  on  peut  constater  que  le  temps  de  calcul  devient  très  important,  Pour  cela,  on  peut  appliquer  les  masques  dans  l’espace  des  fonctions  de  Fourier  qui  rendront  les  calculs  beaucoup  plus  rapides  car  il  existe  un  algorithme  de  calcul  de  la  transformée très rapide (FFT). 

La  fréquence  dans  une  image  représente  la  variation  de  l’intensité  des  pixels  de  l’image,  les  basses  fréquences  (correspondent  à  des  changements  d’intensité  lents)  représentent  les  régions  homogènes  et  floues,   tandis   que   les   hautes   fréquences   (correspondent   à   des   changements   d’intensité   rapides)   représentent les contours et les  changements brusques d’intensité. 

**Référence** https://elearn.univ-ouargla.dz/2013-2014/courses/TI/document/Cours/Image_Chap04.pdf?cidReq=TI

<img src="https://plus.maths.org/content/sites/plus.maths.org/files/articles/2017/carola/Plus-FT.jpg">

<img src="https://plus.maths.org/content/sites/plus.maths.org/files/articles/2017/carola/Plus45-FT.jpg">


Dans ce partie de TP on abordera les problèmes de représentation et de filtrage
(dans le domaine spatial et dans le domaine transformé) (passe-bas et passe haut).  

On utilisera donc à nouveau Python, avec les bibliothèques scientifiques scipy et numpy, ainsi que les modules `scipy.signal` et `scipy.ndimage.` Quelques indications sur les commandes et les fonctions
utiles sont fournis dans l'énoncé. Pour le reste, il vous faudra utiliser l'enseignant
et la documentation en ligne.


Pour faciliter vos tâches, vous pouvez vous servire des fonctions suivants:

  ` 
    rect2 -- crée un rectangle centré en 2D
    filtre_passebande_2d -- crée un filtre passe-bande en 2D (dans le domaine fréquentiel)
    showfft2 -- affiche une image d'une TF 2D, correctement centrée et normalisée
    mesh -- affiche une représentation ``3D'' d'un objet
`

**Rappel**

Pour lire un fichier image, vous utiliserez la fonction `imread`

Pour afficher une image en niveaux de gris, vous pourrez utiliser 

``
        imshow(S,cmap='gray',origin='upper')
``

In [ ]:
from __future__ import division
from __future__ import print_function
import cv2
from pylab import *
import matplotlib.pyplot as plt
from scipy import ndimage as ndi
from scipy import signal
from numpy.fft import fft, ifft, fft2, ifft2
from scipy.ndimage import convolve as convolvend
# pour les représentations 3D
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [ ]:
## Fonctions utils
# ============

def isodd(num):
        return num & 1 and True or False
# or num%2==1        

## 
#def rect2(L,N):
#    """ Rend un rectangle de taille (2L+1)x(2L+1) \n
#    centré sur une dimension totale de NxN"""
#    x=zeros((N,N))
#    milieu=(int((N+1)/2),int((N+1)/2)) if isodd(N) else (int((N)/2+1),int((N)/2+1))
#    print (milieu)
#    x[milieu[0]-L:milieu[0]+L+1,milieu[1]-L:milieu[1]+L+1]=1    
#    return x

## 
def rect2(L,N):
    """ Rend un rectangle de taille (2L+1)x(2L+1) \n
    centré sur une dimension totale de NxN"""
    x=zeros((N,N))
    milieu=((N+1)/2,(N+1)/2) if isodd(N) else ((N)/2+1,(N)/2+1)
    print(milieu)
    x[int(milieu[0])-L:int(milieu[0])+L+1,int(milieu[1])-L:int(milieu[1])+L+1]=1    
    return x


def filtre_passebande_2d(centre,largeur,dimension_totale):    
    """ Retourne la réponse en fréquence d'un filtre passe-bande
    centré sur `centre`, de demi-largeur **largeur** et symétrisé 
    en fréquence (le zéro est placé au centre de l'image en f)\n
    centre : fréquence centrale du filtre passe-bande (en points) -- les fréquences 
    normalisées correspondantes sont ainsi centre/(dimension_totale/2) \n
    largeur :  demi-bande de fréquence (en points) du filtre passe-bande\n
    dimension_totale : dimension de l'image\n 
    Auteur : jfb"""
    N=dimension_totale    
    milieu=((N+1)/2,(N+1)/2) if isodd(N) else ((N)/2+1,(N)/2+1)
    # 
    H=zeros((N,N))    
    H[int(milieu[0])+centre[0]-largeur:int(milieu[0])+centre[0]+largeur+1,int(milieu[1])+centre[1]-largeur:int(milieu[1])+centre[1]+largeur+1]=1    
    centre=-array(centre)  # Le symétrique en fréquence
    H[int(milieu[0])+centre[0]-largeur:int(milieu[0])+centre[0]+largeur+1,int(milieu[1])+centre[1]-largeur:int(milieu[1])+centre[1]+largeur+1]=1    
    return H
    
## représentation en fréquence 2D
def showfft2(X,Zero='uncentered', Freq='normalized', num=None,cmap='hot'):
    """Affiche une image, dans le domaine de Fourier 2D, \n
    la représentation étant centrée et en fréquences réduites\n
    **/!\\  la fonction *showfft2* **ne calcule pas la TF**. Les données passées sont sensées être des données fréquentielles\n
    Paramètres :
    -----------
        X : données 2D dans le domaine de Fourier \n
        Num: numéro de figure (optionnel)\n
        Zero='uncentered' par défaut, la TF passée est supposée non centrée et un fftshift est appliqué\n
        Freq='normalized' par défaut; sinon on trace en points \n
    ``Auteur: jfb``
    """
    N,M=shape(X)
    figure(num)
    if Zero!='centered':
        X=fftshift(X)
    if Freq=='normalized':
        extent=(-0.5, 0.5, -0.5, 0.5)
    else:
        extent=(-N/2, N/2, -N/2, N/2)
    im=imshow(X,aspect='auto',origin='lower', extent=extent, cmap=cmap)    
    if Freq=='normalized':   
        ticks=arange(-0.5,0.6,0.1)
        plt.xticks(ticks)
        plt.yticks(ticks)
    colorbar()
  

   
def mesh(obj3D,numfig=None,subplt=(1,1,1),cmap='hot'):
    """
    Emule le mesh de matlab : trace 
    une représentaion en 2.5D \n
    de l'objet obj3D
    Auteur: jfb
    """
    fig = plt.figure(numfig)
    ax = fig.add_subplot(subplt[0],subplt[1],subplt[2], projection='3d')
    m,n=shape(obj3D)
    mm=range(m)
    nn=range(n)
    X,Y=meshgrid(mm,nn)
    ax.plot_surface(X, Y, obj3D, rstride=max([round(m/50), 1]), cstride=max([round(n/50), 1]), cmap=cmap)
    
def saltpepper(percent=85,shape=None):
        s=numpy.random.random(size=shape)
        d=where(s>percent/100)
        out=ones(shape=shape)
        out[d]=0
        return out


# Représentation fréquentielle

## Une petite sinusoïde

### **[Exercice 1]** A vous de jouer:   
1. Créez une image $N\times N$ (par exemple $512\times 512$) pour la quelle:
  * Affectez les valeurs des pixels selon l'équation $f(x,y) = \sin(2\pi (f_x x+f_y y))$ avec $f_x$ et $f_y$ choisis entre 0.02 à 0.2

2. Visualisez l'image résultante en couleurs (map de chaleur) en utilisant 

```python
                        imshow(image,cmap='hot',origin='upper')
```
<img src="img/sinWave.png" width="150" height="150"> 

3. Utilisez la fonction `showfft2` pour calculer et visualiser la representation fréquentielle de l' image crée
```python 
                           showfft2(abs(fft2(image)),num=4)
```

            <img src="img/fft_sinWave.png" width="150" height="150"> 


4. Avec 3 valeurs differents de $f_x$ et $f_y$ de votre choix, visualiser la représentation fréquentielle de l'image crée, Que observez vous?
    
5. Chargez l'image de Barbara, par `imread('barbara.png')` et visualisez la.
6. Visualisez la représentation en fréquence `showfft2` , en échelle logarithmique (prendre `log(abs())`!)

    * pour visualiser la représentation fréquenctiel (comme l'exemple en-dessus) avec une map des coleurs utilsiez
```python
            imshow(image,cmap='hot',origin='upper')
```

<img src="img/barbara.png" width="150" height="150"> 
<img src="img/fft_barbara.png" width="150" height="150"> 

In [ ]:
# > Emplacement exercice <


# Filtrage en fréquence 

On peut réaliser une transformée de Fourier sur une image en utilisant la méthode de transformée de Fourier rapide de Numpy en dimension 2 : numpy.fft.fft2.

## Filtre passe-bas

In [ ]:
from pylab import *

B=imread('img/barbara.png')
figure(1)
imshow(B,cmap='gray',origin='upper')
colorbar()


L=40
H=rect2(L,512)
showfft2(H,Zero='centered') ## !    
title('Le rectangle en fréquence')
# Filtrage
Bf_filtered=fft2(B)*fftshift(H)
showfft2(log(abs(Bf_filtered)))
title('la TF2D de l''image filtrée')

### **[Exercice 2]** A vous de jouer:   
1. Filtrez cette image à l'aide d'un filtre de réponse en fréquence rectangulaire (utilisez la fonction `rect2`), pour des rectangles de demi-largeur (L) : 80 ,100. 
2. Visualisez les différentes images obtenues (filtrées passe-bas), pour ça vous allez besoin de retrouver les images filtrées en utilisant l'inverse de la transformé de Fourier
```python
            imshow(real(ifft2(imagefft_filtered)),cmap='gray',origin='upper')
```
avec `imagefft_filtered` la TF2D de l''image filtrée

3. Visualisez par une image les différences entre l'image de départ et l'image filtrées. 

**illustration des résultats**


<img src="img/barbara.png" width="150" height="150"> (a)
<img src="img/recovered_barbara.png" width="150" height="150"> (b)
<img src="img/diffrence_img.png" width="150" height="150"> (c)



(a) l'image original
(b) l'image filtrée
(c) l'image de différence entre l'image initiale et sa filtrée

In [ ]:
# > Emplacement exercice <


## Filtre passe-bande

Pour construire une réponse en fréquence qui élimine sélectivement les fréquences situées autour des points selectés comme (46,54) par exemple sur un voisinage de  $\pm$ 10 points de dimension. Pour ce faire, vous utiliserez le filtre passe-bande `filtre_passebande_2d` et créerez un réjecteur de fréquence par `1-filtre_passebande_2d`.

In [ ]:
# Filtrage à encoche
B=imread('img/barbara.png');
Bf=fft2(B)
showfft2(log(abs(Bf)),Zero='uncentered',Freq='unnorm') ## !  
title("TF2D de l'image initiale")
## figure(3)
X=filtre_passebande_2d((46,54),10,512) 
# Attention ; quand on adresse le tableau par A(x,y), les "x" ce sont les 
# lignes (1er indice) et les "y" les colonnes, donc pour les "coordonnées", il faut 
# échanger les deux indices

showfft2(X,Zero='centered',Freq='unnorm') 
title('Filtre passe bande (fréquences)')

### **[Exercice 3]** A vous de jouer: 

1. - Construisez une réponse en fréquence qui élimine sélectivement les fréquences situées autour des points (-70,79), par exemple sur un voisinage de  $\pm$ 10 points de dimension. Pour ce faire, vous utiliserez le filtre passe-bande `filtre_passebande_2d`.
2. créerez un réjecteur de fréquence par `1-filtre_passebande_2d`.
3. Appliquez ce filtre à l'image de départ et visualisé l'image filtrée

**illustration de résultat**

<img src="img/passe-band_barb.png" width="150" height="150"> 
4. Visualisez également l'image différence.

**illustration de résultat**

<img src="img/out.png" width="150" height="150"> 


In [ ]:
# > Emplacement exercice <


# II. Résolution et Quantification
[référence](http://images.math.cnrs.fr/le-traitement-numerique-des-images.html?lang=fr)

## II. a. La résolution d’une image

Afin de réduire la place de stockage d’une image, on peut réduire sa résolution, c’est-à-dire diminuer le nombre de pixels.

La façon la plus simple d’effectuer cette réduction consiste à supprimer des lignes et des colonnes dans l’image de départ.

La figure suivante montre ce que l’on obtient si l’on retient une ligne sur 4 et une colonne sur 4.

<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section1-original-0f409.png" width="150" height="150">

On a ainsi divisé par 4×4=16 le nombre de pixels de l’image, et donc également réduit par 16 le nombre de bits nécessaires pour stocker l’image sur un disque dur.

La figure suivante montre les résultats obtenus en gardant de moins en moins de lignes et de colonnes. Bien entendu, la qualité de l’image se dégrade vite.

<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section2-subsample-2-4b49a.png" width="150" height="150">(a)
<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section2-subsample-4-0295c.png" width="150" height="150">(b)
<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section2-subsample-8-2ddeb.png" width="150" height="150">(c)
<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section2-subsample-16-c3b9c.png" width="150" height="150">(d)

(a) Une ligne/colonne sur 2, (b) Une ligne/colonne sur 4, (c) Une ligne/colonne sur 8, (d) Une ligne/colonne sur 16

### **[Exercice 4]** A vous de jouer: 
1. écrivez la fonction que permet de réduire la résolution d'une image d'entrée si on retien que un pixel sur 2, 4, 8 et 16.
2. visualisez vous résultats.
3. comparer la taille de chaque image résultant.  

In [ ]:
# > Emplacement exercice <


## II. b. Quantifier une image

Une autre façon de réduire la place mémoire nécessaire pour le stockage consiste à utiliser moins de nombres entiers pour chaque valeur.

On peut par exemple utiliser uniquement des nombres entiers entre 0 et 3, ce qui donnera une image avec uniquement 4 niveaux de gris.

On peut effectuer une conversion de l’image d’origine vers une image avec 3 niveaux de valeurs en effectuant les remplacements :

    * les valeurs dans 0,1,…,63 sont remplacées par la valeur 0,
    * les valeurs dans 64,65,…,127 sont remplacées par la valeur 1,
    * les valeurs dans 128,129,…,191 sont remplacées par la valeur 2,
    * les valeurs dans 192,193,…,255 sont remplacées par la valeur 3.
Une telle opération se nomme quantification.

La figure suivante montre l’image résultante avec 4 niveaux de gris. Les 4 valeurs sont affichées en utilisant 4 niveaux de gris allant du noir au blanc.



<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section3-quantize-16-9fcea.png" width="150" height="150"> 16 niveaux de gris

Nous avons déjà vu que l’on pouvait représenter toute valeur entre 0 et 255 à l’aide de 8 bits en utilisant l’écriture binaire. De façon similaire, on vérifie que toute valeur entre 0 et 3 peut se représenter à l’aide de 2 bits. On obtient ainsi une réduction d’un facteur 8/2=4 de la place mémoire nécessaire pour le stockage de l’image sur un disque dur.

La figure suivante montre les résultats obtenus en utilisant de moins en moins de niveaux de gris.

<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section3-quantize-16-9fcea.png" width="150" height="150">(a)
<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section3-quantize-4-c8e17.png" width="150" height="150"> (b)
<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section3-quantize-3-833fc.png" width="150" height="150"> (c)
<img src="http://images.math.cnrs.fr/local/cache-vignettes/L256xH256/section3-quantize-2-170a7.png" width="150" height="150"> (d)

(a) 16 niveaux de gris, (b) 4 niveaux de gris, (c) 3 niveaux de gris, (d) 2 niveaux de gris

Tout comme pour la réduction du nombre de pixels, la réduction du nombre de niveaux de gris influe beaucoup sur la qualité de l’image. Afin de réduire au maximum la taille d’une image sans modifier sa qualité, on utilise des méthodes plus complexes de compression d’image. La méthode la plus efficace s’appelle JPEG-2000. Elle utilise la théorie des ondelettes. Pour en savoir plus à ce sujet, vous pouvez consuler cet article [d’Erwan Le Pennec](http://images.math.cnrs.fr/Compression-d-image.html?lang=fr).



### **[Exercice 4]** A vous de jouer: 
1. écrivez la fonction que permet de réduire la résolution d'une image d'entrée si on restraint les valeur des pixels à 16, 4 et niveau de gris.
2. visualisez vous résultats.
3. comparer la taille de chaque image résultant.

In [ ]:
# > Emplacement exercice <
